In [1]:
# Imports
import pandas as pd
import gensim
import numpy as np
import nltk
import re

from helpers  import *

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

#nltk.download('wordnet')

np.random.seed(2018) # set random seed


C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import spacy
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()

#import parser able to cut sentences into english words

def tokenize(text):
    '''
    cuts the sentences into words, performs also specific operations on URLs and @, plus lowercasing
    '''
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
            #PROBABLY BETTER THIS ONE BUT LDA CRASHES DUE TO THE HUGE NUMBER OF WORDS WITH 1 OCCURRENCE (NEED TO AGGREGATE)
            #lda_tokens.append(token.string[1:].lower())
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [3]:
token = parser('La CAcca è blu')
for tok in token:
    print(tok.string[1:].lower())

a 
acca 
 
lu


In [4]:
# Load data
tweets1 = pd.read_csv(access_folder('data') + 'IRAhandle_tweets_1.csv')

# selecting content columns for subject categorization (all should be in english)
content = tweets1[tweets1.language == 'English'].content

content[1]

'Marshawn Lynch arrives to game in anti-Trump shirt. Judging by his sagging pants the shirt should say Lynch vs. belt https://t.co/mLH1i30LZZ'

In [5]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

def get_lemma(word):
    '''
    To be understood better, it should transform words into their roots
    '''
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer

def get_lemma2(word):
    '''
    To be understood better, it should transform words into their roots. DIFFERENT VERSION
    '''
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
nltk.download('stopwords')

#define set of stopwords to be ignored 

en_stop = set(nltk.corpus.stopwords.words('english'))
print(en_stop)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\utente\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'she', 'll', 'isn', 'ourselves', 'other', 'wouldn', 'who', 'their', 'be', 'yourselves', 'the', 'during', 'ain', 'into', 'haven', 'him', 'after', 'not', 'that', 'above', 'here', 'where', 'itself', 'them', 'about', "wasn't", 'they', 'having', 'only', "didn't", 'theirs', "you're", "should've", 'now', 'does', 'with', 'we', "it's", 'whom', 'ours', 'each', 'most', "don't", 'because', 'while', 'to', 'couldn', 'or', 'ma', 'hadn', 'how', 'has', 'from', 's', 'doesn', 'don', 'as', 'this', 'were', 'very', "isn't", 'hasn', "she's", 'few', 'have', 'me', 'an', "wouldn't", 'a', "couldn't", 'in', 'between', 'myself', 'more', 'mightn', 'you', 'just', 'off', 'too', 'why', 'should', "doesn't", 'both', 'it', 'for', 'i', 'd', 'shan', 're', "you'd", 'own', 'again', "won't", 'been', 'by', 'over', 'there', "haven't", 'won', "mustn't", 'when', 'down', 

In [7]:
def prepare_text_for_lda(text, filter_size = 3):
    
    '''
    Collection of all function described above + filtering for short words (default > 3 letters)
    '''
    
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > filter_size]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma2(token) for token in tokens]
    return tokens

In [8]:
#apply transformation to every tweets to fit the lda datatype

import random
text_data = []
for line in content:
    tokens = prepare_text_for_lda(line)
    if random.random() > .9999:
        print(tokens)
        text_data.append(tokens)

['whole', 'effort', 'last', 'decade', 'toward', 'minority', 'teacher', 'recruitment', 'unheralded', 'victory']
['football', 'coach', 'barry', 'bennell', 'court', 'child', 'sexual', 'offence']
['black', 'panther', '-jean', 'grae', 'darkest', 'night', 'ever']
['missed', 'early', 'voting', 'vote', 'tuesday', 'april', '18th', 'votejonossoff', 'flipthe6th']
['epmd', 'song']
['igetalittleweirdabout', 'seeing', 'picture']
['fact', 'check', 'price', 'scheduled', 'drop', 'well', 'took', 'office', 'dishonest', 'claim']
['nothing', 'say', 'wish', 'younger', 'quite', 'like', 'people', 'using', 'emojis', 'twitter', 'name']
['ifiweretedcruz', 'would', 'become', 'killer', 'presidential', 'campaign', 'failed']
['wikileaksemails', 'donald', 'trump', 'pick', 'jeff', 'session', 'attorney', 'general', 'clinton', 'foundation', 'indictment', 'imminent']
['nostradamus', 'prediction', 'putin', 'trump', 'stop', 'wwiii']
['SCREEN_NAME', 'SCREEN_NAME', 'SCREEN_NAME', 'SCREEN_NAME', 'twinfallsidaho', 'either', 'a

In [9]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)

#Build corpus of frequency occurrencies

corpus = [dictionary.doc2bow(text) for text in text_data]

In [15]:
'''
Train the model: 
-NUM_TOPICS: must be tuned with human judgement;

-alpha     : tunes the number of topics that could generate one word,
           the lower alpha the easier it is to get a word belonging only to few topics
'''

NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

...

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsum

In [17]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.004*"legalizing" + 0.004*"houston" + 0.004*"horse" + 0.004*"racing"')
(1, '0.047*"health" + 0.047*"trump" + 0.047*"donald" + 0.047*"politics"')
(2, '0.051*"interference" + 0.051*"goodsaltydog" + 0.051*"seanhannity" + 0.051*"show"')
(3, '0.045*"pick" + 0.045*"imminent" + 0.045*"clinton" + 0.045*"general"')
(4, '0.072*"igetalittleweirdabout" + 0.072*"seeing" + 0.072*"picture" + 0.003*"racing"')
(5, '0.047*"hand" + 0.047*"grae" + 0.047*"lying" + 0.047*"hillary"')
(6, '0.080*"SCREEN_NAME" + 0.080*"sexysports" + 0.041*"houston" + 0.041*"rocket"')
(7, '0.027*"SCREEN_NAME" + 0.027*"whenwillpeoplewakeup" + 0.027*"18th" + 0.027*"another"')
(8, '0.004*"legalizing" + 0.004*"houston" + 0.004*"horse" + 0.004*"racing"')
(9, '0.030*"court" + 0.030*"football" + 0.030*"child" + 0.030*"coach"')
(10, '0.027*"tell" + 0.027*"teen" + 0.027*"problem" + 0.027*"right"')
(11, '0.063*"invasion" + 0.063*"home" + 0.063*"leaf" + 0.063*"fighting"')
(12, '0.392*"SCREEN_NAME" + 0.047*"either" + 0.024*"thing" + 

In [ ]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [16]:
import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)

pyLDAvis.display(lda_display)

C:\Users\utente\Anaconda3\envs\ada\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type 'complex' is not JSON serializable

__TEST__


In [ ]:
text_data = [['apples', 'oranges', 'avocados'],\
             ['machinegun', 'pistols'],\
             ['flu', 'cold', 'healthy']]
text_data

In [ ]:
# Create dictionary to know the frequency of words
dictionary = gensim.corpora.Dictionary(text_data)


# Create bag-of-words
bow_corpus = [dictionary.doc2bow(doc) for doc in text_data]
bow_corpus

ldamodel = gensim.models.ldamodel.LdaModel(bow_corpus, num_topics = 3, id2word=dictionary)

In [ ]:
bow_corpus

In [ ]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))